# LU- и LUP-разложение

#### Морозов Евгений Александрович НПМбд-01-19

Имеем СЛАУ A * x = b. 
Если матрица A не вырождена, можно искать решение методом LU- и LUP- разложений.

In [149]:
import numpy as np
import scipy.linalg

## LU-разложение

LU-разложение — это представление матрицы A в виде A=L*U, где L — нижнетреугольная матрица с еденичной диагональю, а U — верхнетреугольная матрица. LU-разложение является модификациеё метода Гаусса. 

In [150]:
def LU(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=float)
    
    for i in range(n):
        L[i + 1:, i] = U[i + 1:, i] / U[i, i]
        U[i + 1:] = (U[i + 1:] - L[i + 1:, i, np.newaxis] * U[i])
            
    return L, U

Проверка: A = L * U

In [151]:
A1 = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 100]],
             dtype=float)

print('A1:')
print(A1)
print()

L, U = LU(A1)

print('L * U')
print(L.dot(U))
print()

A1:
[[  1.   2.   3.]
 [  4.   5.   6.]
 [  7.   8. 100.]]

L * U
[[  1.   2.   3.]
 [  4.   5.   6.]
 [  7.   8. 100.]]



### LUP-разложение

На диагонали матрицы A может стоять нуль, в этом случае у нас не получится делить на него в методе LU. Тогда мы можем в изначальной матрице переставлять строки и запоминать перестановки в матрицу перестановок P.

In [152]:
def LUP(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    P = np.eye(n, dtype=np.double)
    
    for i in range(n):
        for k in range(i, n): 
            if ~np.isclose(U[i, i], 0.0):
                break
            U[[k, k+1]] = U[[k+1, k]]
            P[[k, k+1]] = P[[k+1, k]]
            
        L[i + 1:, i] = U[i + 1:, i] / U[i, i]
        U[i + 1:] = (U[i + 1:] - L[i + 1:, i, np.newaxis] * U[i])
        
    return P, L, U


Проверка: A = P.T * L * U. Добавим ноликов:

In [153]:
A2 = np.array([[0, 2, 3],
               [4, 0, 0],
               [7, 8, 10]],
              dtype=float)



P, L, U = LUP(A2)

print('A2')
print(A2)
print()

print('P.T * L * U')
print(P.T.dot(L.dot(U)))
print()

A2
[[ 0.  2.  3.]
 [ 4.  0.  0.]
 [ 7.  8. 10.]]

P.T * L * U
[[ 0.  2.  3.]
 [ 4.  0.  0.]
 [ 7.  8. 10.]]



Решаем систему L * y = b

In [154]:
def forward_substitution(L, b):
    n = A.shape[0]
    y = np.zeros_like(b, dtype=np.double)
    y[0] = b[0] / L[0, 0]
    
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
        
    return y

Решаем систему U * x = y

In [155]:
def back_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=np.double);

    x[-1] = y[-1] / U[-1, -1]

    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]
        
    return x

### Решаем СЛАУ

In [156]:
# вектор правой части
b = np.array([10, 20, 33], dtype=float)

In [157]:
def LU_solve(A, b):
    L, U = LU(A)
    y = forward_substitution(L, b)
    
    return back_substitution(U, y)

In [158]:
def LUP_solve(A, b):
    P, L, U = LUP(A)
    y = forward_substitution(L, np.dot(P.T, b))
    
    return back_substitution(U, y)

Решим A1 * x = b:

In [160]:
print(LU_solve(A1, b))
print(np.linalg.solve(A1, b))

[-3.3003663   6.6007326   0.03296703]
[-3.3003663   6.6007326   0.03296703]


Решим A2 * x = b:

In [162]:
print(LUP_solve(A2, b))
print(np.linalg.solve(A2, b))

[  5.  -26.5  21. ]
[  5.  -26.5  21. ]


Вывод: 

Используем numpy, если хотим быстро работать с матрицами, т.к. numpy написан на С (но не используем numpy, если нам надо в матрицы добавлять элементы разных типов).

Удобно использовать LU- и LUP-разложение, если надо считать решение для одной и той же матрицы, но с разной правой частью, так как вектор b в данном методе ипользуется только на заключительном этапе.